In [60]:
import pandas as pd
import json
random_state = 0
with open('fewrel_val.json') as fp:
    data_val = json.load(fp)
with open('fewrel_train.json') as fp:
    data_train = json.load(fp)
l1 = [[data['tokens'], key] for key in data_train.keys() for data in data_train[key] ]
l2 = [[data['tokens'], key] for key in data_val.keys() for data in data_val[key] ]
df = pd.DataFrame(l1+l2, columns=['words', 'label'])
df['sentence'] = df['words'].apply(lambda l: " ".join(l))
df = df[['sentence', 'label']]
df.columns = ['text', 'label']
len(data_train.keys()), len(data_val.keys())

(64, 16)

In [61]:
df_train, df_dev_test = train_test_split(df, test_size=0.2, stratify=df.label, 
                                     shuffle=True, random_state=random_state)

In [62]:
from sklearn.model_selection import train_test_split
df_dev, df_test = train_test_split(df_dev_test, test_size=0.5, stratify=df_dev_test.label, 
                                     shuffle=True, random_state=random_state)

In [63]:
df_train.shape, df_test.shape, df_dev.shape

((44800, 2), (5600, 2), (5600, 2))

In [64]:
df_train.to_csv('ori_train.tsv', sep='\t', index=False)
df_dev.to_csv('ori_dev.tsv', sep='\t', index=False)
df_test.to_csv('ori_test.tsv', sep='\t', index=False)
df.to_csv('all.tsv',sep='\t',index=False)

In [2]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    """
    string = re.sub(r"[^A-Za-z0-9:(),!?\'\`]", " ", string)
    string = re.sub(r" : ", ":", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

def clean_str_vn(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    """
    string = re.sub(r"[~`@#$%^&*-+]", " ", string)    
    def sharp(str):
        b = re.sub('\s[A-Za-z]\s\.', ' .', ' '+str)
        while (b.find('. . ')>=0): b = re.sub(r'\.\s\.\s', '. ', b)
        b = re.sub(r'\s\.\s', ' # ', b)
        return b
    string = sharp(string)
    string = re.sub(r" : ", ":", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [9]:
text = 'alekseyenko , alekseenko , alexeenko \( \) is a russified form of the ukrainian surname oleksienko derived from the first name oleksiy , \( from greek alexius russian:aleksey \)'
text = text.replace("\(","")
text = text.replace("\)","")
print(text)

alekseyenko , alekseenko , alexeenko   is a russified form of the ukrainian surname oleksienko derived from the first name oleksiy ,  from greek alexius russian:aleksey 


In [2]:
import numpy as np
a = np.load('labels.npy')
print(a)
print(len(a))

['P403' 'P1435' 'P206' 'P39' 'P127' 'P155' 'P140' 'P241' 'P412' 'P106'
 'P264' 'P3450' 'P355' 'P400' 'P1408' 'P463' 'P1923' 'P460' 'P706' 'P57'
 'P413' 'P740' 'P306' 'P40' 'P17' 'P4552' 'P176' 'P1303' 'P150' 'P59'
 'P410' 'P159' 'P58' 'P175' 'P527' 'P123' 'P31' 'P495' 'P131' 'P407'
 'P156' 'P178' 'P1344' 'P1001' 'P641' 'P27' 'P674' 'P466' 'P177' 'P25'
 'P361' 'P3373' 'P364' 'P135' 'P710' 'P551' 'P276' 'P22' 'P750' 'P6'
 'P118' 'P101' 'P102' 'P449' 'P1346' 'P136' 'P1411' 'P1877' 'P2094' 'P26'
 'P105' 'P137']
72


In [56]:
import csv
import numpy as np
import re
load_data_path = 'all.tsv'
data_path_test = 'test.tsv'
labels = []
texts = []
len_x = 0
sum_l = 0
max_l = 0
with open(load_data_path,'r',encoding = "utf-8") as f:
    reader = csv.reader(f,delimiter = "\t")
    for row in reader:
        text = row[0]
        label = row[1]
        if text == 'text':
            continue

        text.replace("\n","")
        texts.append(text)
        labels.append(label)
        l = len(text.split())
        sum_l += l
        if l > max_l:
            max_l = l
        if l > len_x:
            len_x = l
        
print(sum_l / len(texts))
print(len_x)
print(max_l)
print(len(np.unique(np.array(labels))))
dicts = {}

def gen_data(text):
    return text.lower().split()
for t in texts:
    s = gen_data(t)
    for w in s:
        dicts[w] = dicts.get(w,0) + 1 

24.935964285714284
36
36
80


In [2]:
import csv
import numpy as np
import re
load_data_path = 'dev.tsv'
data_path_test = 'test.tsv'
labels = []
texts = []
with open(load_data_path,'r',encoding = "utf-8") as f:
    reader = csv.reader(f,delimiter = "\t")
    for row in reader:
        text = row[0]
        label = row[1]
        if text == 'text':
            continue
        texts.append(text)
        labels.append(label)
labels = np.array(labels)
label_list = np.unique(labels)
print(label_list)
print(len(labels))
for label in label_list:
    print(len(labels[labels == label]))

['<UNK>' 'P101' 'P1303' 'P140' 'P17' 'P176' 'P206' 'P276' 'P306' 'P3450'
 'P39' 'P403' 'P407' 'P4552' 'P460' 'P463' 'P495' 'P57' 'P58' 'P800'
 'P931']
5600
4200
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
